**What does this notebook do?**
- Load the exported CGM values from NutriSense
- Print out what days are included in the dataset
- Pair down data to only one day, include CGM values, meals and exercise
- Smooth CGM data and interpolate missing values
- Pull in Garmin step information and "run activities" and plot them
- Calculate key metrics for that day, both glucose and steps
- Retrieve sleep data from an outside Excel file for that day
- Create a chart of the glucose values, meals, activities, steps, sleep and include some metrics

In [82]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import date
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
from openpyxl import Workbook, load_workbook

# Read in CSV file
df = pd.read_csv('export3.csv')

# Remove "time zone offset" from "occurred_at" column and add new "occurred_at_day" column
df['occurred_at_day'] = df['occurred_at'].apply(lambda x: x[:len(x) - 15])
df['occurred_at'] = df['occurred_at'].apply(lambda x: x[:len(x) - 6])

# Read Excel file
workbook = load_workbook(filename = 'tracking.xlsx')
# Load the sheet with the data I am interested in
todaySheet = workbook['Today']

# Get Garmin Data
# This may not be so great, defaulting to simply retrieving the last 100 activities on Garmin.
# If the day that is plotted is further in the past, this may not work.
numberOfActivities = 100
try:
    # Read UserID and Password from config file
    config  = {}
    with open("config.dat") as myfile:
        for line in myfile:
            name, var = line.partition("=")[::2]
            config[name.strip()] = str(var).strip()
    # Initialize Garmin client with credentials
    client = Garmin(config["uid"], config["password"])
    # Login to Garmin Connect portal
    client.login()
    # Get running activities
    allActivities = client.get_activities(0,numberOfActivities) # 0=start, numberOfActivities=limit  
except (GarminConnectConnectionError, GarminConnectAuthenticationError, GarminConnectTooManyRequestsError,) as err:
    print("Error occured during Garmin Connect Client init: %s" % err)
    quit()
except Exception:
    print("Unknown error occured during Garmin Connect Client init.")

In [83]:
# Print all days with data
daysWithData = df['occurred_at_day'].unique()
print(daysWithData)

['2021-09-10' '2021-09-09' '2021-09-08' '2021-09-07' '2021-09-03'
 '2021-09-02' '2021-09-01' '2021-08-31' '2021-08-30' '2021-08-29'
 '2021-08-28' '2021-08-27' '2021-08-26' '2021-08-25' '2021-08-24'
 '2021-08-23' '2021-08-22' '2021-08-21' '2021-08-20' '2021-08-19'
 '2021-08-18' '2021-08-17' '2021-08-16' '2021-08-15' '2021-08-14'
 '2021-08-13' '2021-08-12' '2021-08-11' '2021-08-10' '2021-08-09'
 '2021-08-08' '2021-08-07' '2021-08-06' '2021-08-05']


In [84]:
# See: https://plotly.com/python/multiple-axes/
maxColumns = 3
maxRows = 3
columnInPlot = 1
rowInPlot = 1
fig = make_subplots(rows=maxRows, cols=maxColumns,
            specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}],
            [{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}],
            [{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}]],
            horizontal_spacing = 0.1, vertical_spacing = 0.1)

# Go through the data and create a plots for every breakfast event in those days
for y in range(1, 13):
    df1 = df[df['occurred_at_day']==daysWithData[y]]
    day = daysWithData[y]

    # Create a datasets just with glucose measurments
    gm = df1[df1['class']=='GlucoseMeasurement']

    # Create a dataset for meals and exercise, sort it
    mealsExercise = df1[((df1['class']=='Meal') | (df1['class']=='ExerciseActivity') )]
    mealsExerciseSorted = mealsExercise.sort_values(by=["occurred_at"], ascending=True)
    allBreakfasts = mealsExerciseSorted[mealsExerciseSorted['time']=='breakfast']
    # loop through all breakfasts of that day
    for index, row in allBreakfasts.iterrows():

        # Get steps of the day from Garmin
        dayOfInterest = datetime.datetime.strptime(day, '%Y-%m-%d').date()
        allDayStepData = client.get_steps_data(dayOfInterest.isoformat())

        # convert garmin data in list form to Pandas dataframe
        dfGarmin = pd.DataFrame.from_dict(allDayStepData)

        # manipulate start time so that it is local (And not GMT)
        dfGarmin['time'] = dfGarmin['startGMT'].apply(lambda x: x[:len(x) - 5])
        dfGarmin['time'] = dfGarmin['time'].apply(lambda x: x[11:])
        offset = dfGarmin["time"][0]
        offsetHour = int(offset.split(':')[0])
        offsetMinutes=int(offset.split(':')[1])
        dfGarmin['time'] = pd.to_datetime(dfGarmin['startGMT'])
        dfGarmin['time'] = dfGarmin['time'].apply(lambda x: x - datetime.timedelta(hours=offsetHour, minutes=offsetMinutes))
        dfGarmin = dfGarmin[dfGarmin.steps != 0]

        # Create a dataset with just 2 columns, and rename columns
        gm_data = gm.filter(['occurred_at', 'value'])
        gm_data.columns = ['time', 'value']

        # turn time column into the index and delete time column
        gm_data['time']= pd.to_datetime(gm_data['time'])
        gm_data.index = gm_data['time']
        del gm_data['time']

        # Add rows and interpolate missing data
        gm_data = gm_data.resample('1T').mean()
        gm_data = gm_data.interpolate(method='cubic')
        
        # Determine what data range to chart
        beginChart = datetime.datetime.strptime(row['occurred_at'], '%Y-%m-%d %H:%M:%S') - datetime.timedelta(hours=2)
        endChart = datetime.datetime.strptime(row['occurred_at'], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=3)
        
        threshold = 120  # this is an arbitrary threshold

        # Calculate statistics on the Garmin data
        numberOfRunningActivitiesToday = 0
        numberOfActivitiesToday = 0
        for i in range(numberOfActivities):
            activity = allActivities[i]
            activityDateTime = activity['startTimeLocal']
            activityDate = datetime.datetime.strptime(activityDateTime, "%Y-%m-%d %H:%M:%S")
            if str(activityDate.date()) == day:
                numberOfActivitiesToday = numberOfActivitiesToday + 1
                if activity["activityType"]["typeKey"] == "running":
                    numberOfRunningActivitiesToday = numberOfRunningActivitiesToday + 1

        # Retrieve sleep data
        for rowInExcel in range(3,55):
            cell = todaySheet[str("A"+str(rowInExcel))]
            # Skip over "empty" cells
            if cell.value is None: continue
            # Assume the cell contains a date value, thus convert it
            cellDate = cell.value.date()
            if str(cellDate) == str(day):
                sleepEnd = todaySheet[str("E"+str(rowInExcel))].value
                sleepBegin = todaySheet[str("F"+str(rowInExcel))].value
                sleepBegin = datetime.datetime.combine(date.min, datetime.datetime.strptime('23:59', '%H:%M').time()) - datetime.datetime.combine(date.min, sleepBegin)
                break

        # first add the glucose measurement data
        fig.add_trace( go.Scatter(x=gm_data.index, y=gm_data.value, mode='lines',line=dict(color="purple")), row=rowInPlot, col=columnInPlot, secondary_y=False)
        
        # add meals and exercise to the chart
        yText = 145
        eventColor = "green"
        for index, row in mealsExerciseSorted.iterrows():

            # If the activity has "run" in the description, don't use it as it is a duplicate from Garmin
            if "run" in str(row['description']).lower(): continue
            # Same for events that have walking in the decription. Simce Garmin Steps can be seen in the chart
            # having those activities is duplicate information
            if "walk" in str(row['description']).lower(): continue

            # Convert the time in pandas to something that we can use as an index for the x-axis placement
            time = datetime.datetime.strptime(row['occurred_at'], '%Y-%m-%d %H:%M:%S')

            # Pick a different color depending on the event
            if (row['class'] == "Meal"): eventColor = "black"
            else: eventColor = "green"

            # Alternate text placement so adjacent text doesn't overlap
            if (yText >= 175): yText = 145
            else: yText = yText + 8

            # draw a vertical line at the time of the meal/exercise
            gmAtThatTime = gm_data.loc[str(time.replace(second=0))].value
            fig.add_shape(type="line", xref="x", yref="y", x0=time, y0=gmAtThatTime, x1=time , y1=yText-2, line_color=eventColor,row=rowInPlot, col=columnInPlot)
            
            # Add text
            fig.add_annotation(text=row['description'], xref="x", yref="y", x=time, y=yText, showarrow=False, font=dict(color=eventColor),row=rowInPlot, col=columnInPlot)

        # Adjust x axis to only show breakfasts
        fig.update_xaxes(range=[beginChart, endChart], title_text=str(dayOfInterest.strftime('%A'))+ ", " +str(day), tickformat='%H:%M',row=rowInPlot, col=columnInPlot)

        # Add Garmin running activities
        for i in range(numberOfActivities):
            activity = allActivities[i]
            # only activities that are of type "running"
            if activity["activityType"]["typeKey"] == "running":
                activityDateTime = activity['startTimeLocal']
                activityDate = datetime.datetime.strptime(activityDateTime, "%Y-%m-%d %H:%M:%S")
                if str(activityDate.date()) == day:
                    # draw a vertical line at the time of the running activity
                    gmAtThatTime = gm_data.loc[str(activityDate.replace(second=0))].value
                    fig.add_shape(type="line", xref="x", yref="y", x0=activityDateTime, y0=gmAtThatTime, x1=activityDateTime , y1=133, line_color="green",row=rowInPlot, col=columnInPlot)
                    # Add text... yes this is specific to kilometers. This may need changes for miles.
                    textDescr = str(activity['activityName']) + " " + str(int(round(activity['distance']/1000))) + "K run"
                    fig.add_annotation(text=textDescr, xref="x", yref="y", x=activityDateTime, y=135, showarrow=False, font=dict(color="green"),row=rowInPlot, col=columnInPlot)

        # Draw a line at the threshold
        fig.add_shape(type="line", xref="x", yref="y",
            x0=gm_data.index[0], y0=threshold, x1=gm_data.index.max(), y1=threshold, line_color="red",row=rowInPlot, col=columnInPlot)

        # Setting primary and secondary y axis titles and ticks
        fig.update_yaxes(range=[0, 180], tick0=0, dtick=20, title_text="mg/dL", secondary_y=False, row=rowInPlot, col=columnInPlot)
        fig.update_yaxes(tick0=0, dtick=500, range=[0,4500], title_text='Steps', secondary_y=True, row=rowInPlot, col=columnInPlot)
        
        # Adding step data to the chart, using the secondary y axis
        fig.add_trace( go.Bar(x=dfGarmin.time, y=dfGarmin.steps, marker_color='red'), secondary_y=True,row=rowInPlot, col=columnInPlot)

        # Hide the legend
        fig.update_layout(showlegend=False)
        
        # Draw morning sleep
        fig.add_shape(type="rect", xref="x", yref="y",
            x0=gm_data.index[0], y0=57, x1=datetime.datetime.strptime(str(day) + " " + str(sleepEnd), '%Y-%m-%d %H:%M:%S'), y1=150,
            line=dict(color="RoyalBlue"),fillcolor="LightSkyBlue",opacity=0.5,line_width=0,row=rowInPlot, col=columnInPlot)
        
        # Draw evening sleep
        fig.add_shape(type="rect", xref="x", yref="y",
            x0=gm_data.index.max(), y0=57, x1=datetime.datetime.strptime(str(day) + " " + str(sleepBegin), '%Y-%m-%d %H:%M:%S'), y1=150,
            line=dict(color="RoyalBlue"),fillcolor="LightSkyBlue",opacity=0.5,line_width=0,row=rowInPlot, col=columnInPlot)
        
        columnInPlot = columnInPlot + 1
        if columnInPlot == maxColumns+1:
            rowInPlot = rowInPlot + 1
            columnInPlot = 1
        if rowInPlot == maxRows+1:
            break
        continue

# Size the chart
fig.update_layout(autosize=False, width=1600, height=1000,margin=dict(l=20, r=20, t=40, b=20))
fig.show()